# KLIMATA ILOILO - CLIMATE VULNERABILITY MAPPING with docstrings

In [2]:
import pandas as pd
import numpy as np

In [5]:
cli_df = pd.read_csv(r"C:\Users\Value Lines\Documents\FINAL_DATASETS\CLIMATE_FINALE.csv")

In [6]:
cli_df.head()

,adm4_pcode,date,ndvi,uuid,no2,co,so2,o3,pm10,pm25,heat_index,pr,adm4_en,brgy_total_area
0,ph063022001,2003-01-01,0.47,cairq61f770,1.21,0.0845,0.41,33.08,20.72,14.15,28.73,6.17,santa cruz,0.5633
1,ph063022001,2003-01-02,0.48,cairq0d5fc0,0.72,0.0940,0.29,37.59,24.16,14.37,26.73,0.00,santa cruz,0.5633
2,ph063022001,2003-01-03,0.44,cairq0d5fc1,0.89,0.0914,0.33,37.41,19.79,12.34,27.12,0.00,santa cruz,0.5633
3,ph063022001,2003-01-04,0.48,cairq0d5fc2,0.71,0.0860,0.31,37.28,20.64,12.88,27.69,0.00,santa cruz,0.5633
4,ph063022001,2003-01-05,0.44,cairq0d5fc3,0.66,0.0862,0.34,35.99,17.62,10.76,27.13,0.00,santa cruz,0.5633


In [9]:
cli_df['date'] = pd.to_datetime(cli_df['date'])

indicators = ['ndvi', 'no2', 'co', 'so2', 'o3', 'pm10', 'pm25', 'heat_index', 'pr']

avg_per_barangay = cli_df.groupby('adm4_pcode')[indicators].mean().reset_index()

print(avg_per_barangay)

      adm4_pcode      ndvi    no2        co       so2         o3       pm10  \
0    ph063022001  0.424081  1.508  0.072363  0.498964  32.245693  19.179458   
1    ph063022003  0.428401  1.508  0.072363  0.498964  32.245693  19.179458   
2    ph063022004  0.562222  1.508  0.072363  0.498964  32.245693  19.179458   
3    ph063022005  0.428401  1.508  0.072363  0.498964  32.245693  19.179458   
4    ph063022006  0.428401  1.508  0.072363  0.498964  32.245693  19.179458   
..           ...       ...    ...       ...       ...        ...        ...   
175  ph063022198  0.428401  1.508  0.072363  0.498964  32.245693  19.179458   
176  ph063022199  0.428401  1.508  0.072363  0.498964  32.245693  19.179458   
177  ph063022200  0.471139  1.508  0.072363  0.498964  32.245693  19.179458   
178  ph063022201  0.428401  1.508  0.072363  0.498964  32.245693  19.179458   
179  ph063022202  0.428401  1.508  0.072363  0.498964  32.245693  19.179458   

          pm25  heat_index        pr  
0    13.1745

In [44]:
avg_df = avg_per_barangay 

In [45]:
avg_df.to_csv("AVG_CLIMATE.csv", index=False)

In [78]:
urban_df = pd.read_csv("URBAN_RISK7.0.csv")

print(urban_df.head())

    adm4_pcode  climate_exposure_score  pop_total      ndvi  infra_index  \
0  ph063022001                0.336261  -0.101248 -0.396651     0.298776   
1  ph063022003                0.337041  -0.477470 -0.344274     0.379013   
2  ph063022004                0.336787   1.276712  1.278059     0.084594   
3  ph063022005                0.337041   0.334198 -0.344274     0.125826   
4  ph063022006                0.337041  -0.540216 -0.344274     0.223698   

   rwi_mean  brgy_distance_to_coast  ndvi_risk  infra_risk  rwi_risk  \
0  1.382495               -0.725769   0.812472    0.592437  0.348195   
1  2.342979                0.165168   0.792488    0.475868  0.051354   
2  1.331623                0.365861   0.173497    0.903601  0.363917   
3  2.235535               -0.056495   0.792488    0.843699  0.084559   
4  1.475280                1.731573   0.792488    0.701509  0.319519   

   coast_risk  pop_risk  urban_risk_index  cluster  risk_level   risk_label  
0    0.925268  0.189198         

In [65]:
urban_df = urban_df[['adm4_pcode', 'climate_exposure_score']]

merge1_df = pd.merge(avg_df, urban_df, on='adm4_pcode', how='inner')

In [66]:
df = pd.read_csv("BRGY_STANDARDIZED.csv")

In [67]:
df = df.drop(columns=['uuid', 'date'])

In [68]:
df['adm4_pcode'] = df['adm4_pcode'].str.lower()

In [69]:
merge_df = pd.merge(df, merge1_df, on="adm4_pcode", how="inner")

In [70]:
merge_df.shape

(180, 17)

In [71]:
merge_df = merge_df.drop(columns = [
    "freq",
    "brgy_total_area",
    "brgy_distance_to_coast",
    "brgy_is_coastal"
])


In [72]:
from shapely import wkt

def safe_load_wkt(value):
    try:
        if pd.isna(value) or not value.strip():
            return None
        return wkt.loads(value)
    except Exception:
        return None  # Skip invalid geometry strings

merge_df["geometry"] = merge_df["geometry"].apply(safe_load_wkt)


In [73]:
merge_df = merge_df[merge_df["geometry"].notnull()]

In [74]:
import pandas as pd
import geopandas as gpd

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(merge_df, geometry="geometry", crs="EPSG:4326")

# Extract boundary coordinates for each polygon
def extract_coords(poly):
    return list(poly.exterior.coords)

gdf["boundary_coords"] = gdf["geometry"].apply(extract_coords)

rows = []
for _, row in gdf.iterrows():
    for lon, lat in row["boundary_coords"]:
        # Copy all columns from the current row
        record = row.to_dict()
        # Replace geometry with coordinates
        record["longitude"] = lon
        record["latitude"] = lat
        # Optional: remove the polygon geometry and coords list (to avoid very large CSV)
        record.pop("geometry", None)
        record.pop("boundary_coords", None)
        rows.append(record)

# Convert to DataFrame
coords_df = pd.DataFrame(rows)

# Save as CSV
coords_df.to_csv("cli_polys3.0.csv", index=False)


In [75]:
coords_df.columns.tolist()

['adm4_pcode',
 'location.adm4_en',
 'ndvi',
 'no2',
 'co',
 'so2',
 'o3',
 'pm10',
 'pm25',
 'heat_index',
 'pr',
 'climate_exposure_score',
 'longitude',
 'latitude']

In [77]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

# --- Load CSV ---
poly_df = pd.read_csv("cli_polys3.0.csv")

# --- Clean column names (GeoJSON-safe) ---
poly_df.columns = poly_df.columns.str.replace(r"[^0-9a-zA-Z_]+", "_", regex=True)

# --- Keep only desired columns (only keep ones that exist) ---
keep_cols = ['adm4_pcode',
 'location_adm4_en',
 'ndvi',
 'no2',
 'co',
 'so2',
 'o3',
 'pm10',
 'pm25',
 'heat_index',
 'pr',
 'climate_exposure_score',
 'longitude',
 'latitude']

# Filter only columns that actually exist in the file
existing_cols = [c for c in keep_cols if c in poly_df.columns]
poly_df = poly_df[existing_cols].copy()

# --- Convert barangay points to polygons ---
records = []
for code, group in poly_df.groupby("adm4_pcode"):
    # group all coordinate pairs
    coords = list(zip(group["longitude"], group["latitude"]))
    if len(coords) > 2:  # valid polygon requires at least 3 points
        row = group.iloc[0].copy()
        row["geometry"] = Polygon(coords)
        records.append(row)
    else:
        print(f" Skipped {code}: not enough points to form polygon")

# --- Create GeoDataFrame ---
if records:
    gdf = gpd.GeoDataFrame(records, crs="EPSG:4326")
    
    # --- Save as GeoJSON ---
    gdf.to_file("iloilo_cli3.0.geojson", driver="GeoJSON")

    print(" GeoJSON successfully saved with these columns:")
    print(gdf.columns.tolist())
    print(f"Total barangays exported: {len(gdf)}")

else:
    print(" No valid polygons were created. Check if your CSV has multiple points per adm4_pcode.")


✅ GeoJSON successfully saved with these columns:
['adm4_pcode', 'location_adm4_en', 'ndvi', 'no2', 'co', 'so2', 'o3', 'pm10', 'pm25', 'heat_index', 'pr', 'climate_exposure_score', 'longitude', 'latitude', 'geometry']
Total barangays exported: 179


In [34]:
print(poly_df.columns.tolist())


['adm4_pcode', 'location_adm4_en', 'date_y', 'pop_count_total', 'adm4_en', 'brgy_total_area_y', 'longitude', 'latitude']


# AMENITY INDEX MAPPING with docstrings

In [58]:
ame_df = pd.read_csv(r"C:\Users\Value Lines\Downloads\FINAL_DATASETS\AMENITY_FINAL.csv")

In [66]:
ame_df.head()

,adm4_pcode,college_nearest,community_centre_nearest,school_nearest,shelter_nearest,town_hall_nearest,university_nearest
0,ph063022075,499.693700,363.721118,196.962495,396.840748,0.000000,457.790233
1,ph063022085,190.538945,294.050675,0.000000,755.295238,524.007659,348.098929
2,ph063022132,249.824455,554.323482,212.821448,600.801916,79.373668,1151.978301
3,ph063022092,554.897237,553.326752,238.790126,1168.852374,0.000000,1370.429642
4,ph063022054,11.836872,332.472393,0.000000,396.790167,309.502693,988.501036


In [60]:
ame_df = ame_df[ame_df['date'] == '2022-01-01']

In [65]:
ame_df = ame_df.drop(columns=["location1.adm4_en", "location1.brgy_total_area" ])

In [64]:
hea_df.head()

,adm4_pcode,brgy_healthcenter_pop_reached_pct_5min,brgy_healthcenter_pop_reached_pct_15min,brgy_healthcenter_pop_reached_pct_30min,hospital_pop_reached_pct_5min,hospital_pop_reached_pct_15min,hospital_pop_reached_pct_30min,rhu_pop_reached_pct_5min,rhu_pop_reached_pct_15min,rhu_pop_reached_pct_30min,adm4_en,brgy_total_area
0,ph063022001,100.00,100.00,100.00,0.00,99.95,100.00,78.41,100.00,100.00,santa cruz,0.5633
1,ph063022003,54.64,54.64,54.64,0.00,54.64,54.64,54.64,54.64,54.64,aguinaldo,0.0652
2,ph063022004,84.84,98.23,98.23,77.88,98.23,98.23,73.20,98.23,98.23,airport (tabucan airport),1.0117
3,ph063022005,99.22,99.22,99.22,0.00,99.22,99.22,99.22,99.22,99.22,alalasan lapuz,0.0729
4,ph063022006,82.85,82.85,82.85,11.01,82.85,82.85,71.84,82.85,82.85,arguelles,0.1224


In [67]:
ame_hea = pd.merge(ame_df, hea_df, on="adm4_pcode", how='inner')

In [70]:
ame_hea

adm4_pcode                                 0
college_nearest                            0
community_centre_nearest                   0
school_nearest                             0
shelter_nearest                            0
town_hall_nearest                          0
university_nearest                         0
brgy_healthcenter_pop_reached_pct_5min     0
brgy_healthcenter_pop_reached_pct_15min    0
brgy_healthcenter_pop_reached_pct_30min    0
hospital_pop_reached_pct_5min              0
hospital_pop_reached_pct_15min             0
hospital_pop_reached_pct_30min             0
rhu_pop_reached_pct_5min                   0
rhu_pop_reached_pct_15min                  0
rhu_pop_reached_pct_30min                  0
adm4_en                                    0
brgy_total_area                            0
dtype: int64

In [134]:
urban_df = pd.read_csv("URBAN_RISK7.0.csv")

In [137]:
urban_df.head()

,adm4_pcode,infra_index,infra_risk
0,ph063022001,0.298776,0.592437
1,ph063022003,0.379013,0.475868
2,ph063022004,0.084594,0.903601
3,ph063022005,0.125826,0.843699
4,ph063022006,0.223698,0.701509


In [136]:
urban_df = urban_df.drop(columns = [
    "climate_exposure_score",
    "pop_total",
    "ndvi",
    "rwi_mean",
    "brgy_distance_to_coast",
    "ndvi_risk",
    "rwi_risk",
    "coast_risk",
    "pop_risk",
    "urban_risk_index",
    "cluster",
    "risk_level",
    "risk_label"
])


In [74]:
urban_df.head()

,adm4_pcode,infra_index,infra_risk
0,ph063022001,0.577563,0.059389
1,ph063022003,-1.120555,0.584960
2,ph063022004,-0.038994,0.250214
3,ph063022005,-0.005273,0.239778
4,ph063022006,-0.415465,0.366733


In [138]:
infra_df = pd.merge(ame_hea, urban_df, on='adm4_pcode', how='inner')

In [139]:
infra_df.head()

,adm4_pcode,college_nearest,community_centre_nearest,school_nearest,shelter_nearest,town_hall_nearest,university_nearest,brgy_healthcenter_pop_reached_pct_5min,brgy_healthcenter_pop_reached_pct_15min,brgy_healthcenter_pop_reached_pct_30min,hospital_pop_reached_pct_5min,hospital_pop_reached_pct_15min,hospital_pop_reached_pct_30min,rhu_pop_reached_pct_5min,rhu_pop_reached_pct_15min,rhu_pop_reached_pct_30min,adm4_en,brgy_total_area,infra_index,infra_risk
0,ph063022075,499.693700,363.721118,196.962495,396.840748,0.000000,457.790233,100.00,100.00,100.00,100.00,100.00,100.00,0.00,100.00,100.00,delgado-jalandoni-bagumbayan,0.0235,0.106237,0.872158
1,ph063022085,190.538945,294.050675,0.000000,755.295238,524.007659,348.098929,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,laguda,0.0537,0.037010,0.972731
2,ph063022132,249.824455,554.323482,212.821448,600.801916,79.373668,1151.978301,94.06,94.06,94.06,42.16,94.06,94.06,94.06,94.06,94.06,progreso-lapuz,0.2741,0.123578,0.846965
3,ph063022092,554.897237,553.326752,238.790126,1168.852374,0.000000,1370.429642,36.64,36.64,36.64,0.00,36.64,36.64,36.64,36.64,36.64,libertad-lapuz,0.0383,0.494559,0.308002
4,ph063022054,11.836872,332.472393,0.000000,396.790167,309.502693,988.501036,100.00,100.00,100.00,100.00,100.00,100.00,48.88,100.00,100.00,edganzon,0.1033,0.063859,0.933724


In [140]:
df = pd.read_csv("BRGY_STANDARDIZED.csv")
df = df.drop(columns=['uuid'])
df['adm4_pcode'] = df['adm4_pcode'].str.lower()
merge1_df = pd.merge(df, infra_df, on="adm4_pcode", how="inner")


In [141]:
merge1_df.head()

,adm4_pcode,date,freq,brgy_total_area_x,brgy_distance_to_coast,brgy_is_coastal,geometry,location.adm4_en,college_nearest,community_centre_nearest,...,hospital_pop_reached_pct_5min,hospital_pop_reached_pct_15min,hospital_pop_reached_pct_30min,rhu_pop_reached_pct_5min,rhu_pop_reached_pct_15min,rhu_pop_reached_pct_30min,adm4_en,brgy_total_area_y,infra_index,infra_risk
0,ph063022084,2003-01-01,S,0.1906,1.081299,False,"POLYGON ((122.564165144 10.720567741, 122.5647...",Our Lady Of Fatima,270.067017,939.179536,...,100.00,100.00,100.00,74.78,100.00,100.00,our lady of fatima,0.1906,0.078199,0.912892
1,ph063022081,2003-01-01,S,0.0434,-0.876514,False,"POLYGON ((122.567721911 10.6961068380001, 122....",Kauswagan,254.069066,604.890348,...,95.71,95.71,95.71,0.00,95.71,95.71,kauswagan,0.0434,0.141381,0.821101
2,ph063022178,2003-01-01,S,0.5440,1.217963,False,"POLYGON ((122.547468591 10.719607266, 122.5480...",Abeto Mirasol Taft South (Quirino Abeto),818.816776,601.559616,...,91.66,91.66,91.66,91.66,91.66,91.66,abeto mirasol taft south (quirino abeto),0.5440,0.085551,0.902211
3,ph063022189,2003-01-01,S,0.1176,-0.593102,False,"POLYGON ((122.561661708 10.697889806, 122.5616...",Villa Anita,492.422668,0.000000,...,100.00,100.00,100.00,0.00,100.00,100.00,villa anita,0.1176,0.080342,0.909778
4,ph063022064,2003-01-01,S,0.8889,1.766896,False,"POLYGON ((122.534580907 10.727787967, 122.5363...",Guzman-Jesena,1273.412539,542.284819,...,88.05,100.00,100.00,87.40,100.00,100.00,guzman-jesena,0.8889,0.084030,0.904420


In [142]:
merge1_df.columns.tolist()

['adm4_pcode',
 'date',
 'freq',
 'brgy_total_area_x',
 'brgy_distance_to_coast',
 'brgy_is_coastal',
 'geometry',
 'location.adm4_en',
 'college_nearest',
 'community_centre_nearest',
 'school_nearest',
 'shelter_nearest',
 'town_hall_nearest',
 'university_nearest',
 'brgy_healthcenter_pop_reached_pct_5min',
 'brgy_healthcenter_pop_reached_pct_15min',
 'brgy_healthcenter_pop_reached_pct_30min',
 'hospital_pop_reached_pct_5min',
 'hospital_pop_reached_pct_15min',
 'hospital_pop_reached_pct_30min',
 'rhu_pop_reached_pct_5min',
 'rhu_pop_reached_pct_15min',
 'rhu_pop_reached_pct_30min',
 'adm4_en',
 'brgy_total_area_y',
 'infra_index',
 'infra_risk']

In [143]:
from shapely import wkt

def safe_load_wkt(value):
    try:
        if pd.isna(value) or not value.strip():
            return None
        return wkt.loads(value)
    except Exception:
        return None  # Skip invalid geometry strings

merge1_df["geometry"] = merge1_df["geometry"].apply(safe_load_wkt)


In [144]:
merge1_df = merge1_df[merge1_df["geometry"].notnull()]

In [145]:
import pandas as pd
import geopandas as gpd

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(merge1_df, geometry="geometry", crs="EPSG:4326")

# Extract boundary coordinates for each polygon
def extract_coords(poly):
    return list(poly.exterior.coords)

gdf["boundary_coords"] = gdf["geometry"].apply(extract_coords)

rows = []
for _, row in gdf.iterrows():
    for lon, lat in row["boundary_coords"]:
        # Copy all columns from the current row
        record = row.to_dict()
        # Replace geometry with coordinates
        record["longitude"] = lon
        record["latitude"] = lat
        # Optional: remove the polygon geometry and coords list (to avoid very large CSV)
        record.pop("geometry", None)
        record.pop("boundary_coords", None)
        rows.append(record)

# Convert to DataFrame
coords_df = pd.DataFrame(rows)

# Save as CSV
coords_df.to_csv("infra_pols3.0.csv", index=False)


In [147]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

# --- Load CSV ---
poly_df = pd.read_csv("infra_pols3.0.csv")

# --- Clean column names (GeoJSON-safe) ---
poly_df.columns = poly_df.columns.str.replace(r"[^0-9a-zA-Z_]+", "_", regex=True)

# --- Keep only desired columns (only keep ones that exist) ---
keep_cols = ['adm4_pcode',
 'date',
 'freq',
 'location_adm4_en',
 'college_nearest',
 'community_centre_nearest',
 'school_nearest',
 'shelter_nearest',
 'town_hall_nearest',
 'university_nearest',
 'brgy_healthcenter_pop_reached_pct_5min',
 'brgy_healthcenter_pop_reached_pct_15min',
 'brgy_healthcenter_pop_reached_pct_30min',
 'hospital_pop_reached_pct_5min',
 'hospital_pop_reached_pct_15min',
 'hospital_pop_reached_pct_30min',
 'rhu_pop_reached_pct_5min',
 'rhu_pop_reached_pct_15min',
 'rhu_pop_reached_pct_30min',
 'adm4_en',
 'infra_index',
 'infra_risk',
 'longitude',
 'latitude']

# Filter only columns that actually exist in the file
existing_cols = [c for c in keep_cols if c in poly_df.columns]
poly_df = poly_df[existing_cols].copy()

# --- Add risk columns (if missing) ---
for col in ['risk_level', 'risk_label']:
    if col not in poly_df.columns:
        poly_df[col] = None  # create blank column
    poly_df[col] = poly_df[col].astype(str)

# --- Convert barangay points to polygons ---
records = []
for code, group in poly_df.groupby("adm4_pcode"):
    # group all coordinate pairs
    coords = list(zip(group["longitude"], group["latitude"]))
    if len(coords) > 2:  # valid polygon requires at least 3 points
        row = group.iloc[0].copy()
        row["geometry"] = Polygon(coords)
        records.append(row)
    else:
        print(f" Skipped {code}: not enough points to form polygon")

# --- Create GeoDataFrame ---
if records:
    gdf = gpd.GeoDataFrame(records, crs="EPSG:4326")
    
    # --- Save as GeoJSON ---
    gdf.to_file("iloilo_infra3.0.geojson", driver="GeoJSON")

    print("✅ GeoJSON successfully saved with these columns:")
    print(gdf.columns.tolist())
    print(f"Total barangays exported: {len(gdf)}")

else:
    print(" No valid polygons were created. Check if your CSV has multiple points per adm4_pcode.")

✅ GeoJSON successfully saved with these columns:
['adm4_pcode', 'date', 'freq', 'location_adm4_en', 'college_nearest', 'community_centre_nearest', 'school_nearest', 'shelter_nearest', 'town_hall_nearest', 'university_nearest', 'brgy_healthcenter_pop_reached_pct_5min', 'brgy_healthcenter_pop_reached_pct_15min', 'brgy_healthcenter_pop_reached_pct_30min', 'hospital_pop_reached_pct_5min', 'hospital_pop_reached_pct_15min', 'hospital_pop_reached_pct_30min', 'rhu_pop_reached_pct_5min', 'rhu_pop_reached_pct_15min', 'rhu_pop_reached_pct_30min', 'adm4_en', 'infra_index', 'infra_risk', 'longitude', 'latitude', 'risk_level', 'risk_label', 'geometry']
Total barangays exported: 179
